In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np

In [2]:
image_size = [224, 224]
vgg = VGG16(input_shape = image_size + [3], weights = 'imagenet', include_top =  False)

In [3]:
for layer in vgg.layers:
    layer.trainable = False

In [4]:
from glob import glob
folders = glob('D:/ST/trainData/Tomato_kaggle_train/*')
folders

['D:/ST/trainData/Tomato_kaggle_train\\Tomato___Bacterial_spot',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Early_blight',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___healthy',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Late_blight',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Leaf_Mold',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Septoria_leaf_spot',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Spider_mites Two-spotted_spider_mite',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Target_Spot',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Tomato_mosaic_virus',
 'D:/ST/trainData/Tomato_kaggle_train\\Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [5]:
outlayer = Flatten()(vgg.output)
outlayer = Dense(4096, activation='relu',name='fc1')(outlayer)
outlayer = Dropout(0.5)(outlayer)
outlayer = Dense(4096, activation='relu',name='fc2')(outlayer)
prediction = Dense(len(folders), activation = 'softmax')(outlayer)
model = Model(inputs = vgg.input, outputs = prediction)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
#이미지 전처리 무작위 변형
train_data_gen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2 \
                                    , zoom_range = 0.2, horizontal_flip = True)

In [10]:
train_set = train_data_gen.flow_from_directory('D:/ST/trainData/Tomato_kaggle_train/', target_size = (224,224), batch_size = 32, class_mode = 'categorical')

Found 18345 images belonging to 10 classes.


In [11]:
test_data_gen = ImageDataGenerator(rescale = 1./255)

In [12]:
test_set = test_data_gen.flow_from_directory('D:/ST/trainData/Tomato_kaggle_test/', target_size = (224,224), batch_size = 32, class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 

path_checkpoint = "Tomato_model_checkpoint.h5"
es_callback = EarlyStopping(monitor="val_loss", min_delta=0, patience=5)
modelckpt_callback = ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

with tf.device("/device:CPU:0"):
    mod = model.fit_generator(train_set,epochs=10, steps_per_epoch=len(train_set) \
                    ,callbacks=[es_callback, modelckpt_callback] \
                    ,validation_data=test_set, validation_steps=len(test_set))

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_7708\1667835563.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mod = model.fit_generator(train_set,epochs=10, steps_per_epoch=len(train_set) \


Epoch 1/10
574/574 [==============================] - ETA: 0s - loss: 2.0196 - accuracy: 0.4351
Epoch 00001: val_loss improved from inf to 1.01292, saving model to Tomato_model_checkpoint.h5
574/574 [==============================] - 1576s 3s/step - loss: 2.0196 - accuracy: 0.4351 - val_loss: 1.0129 - val_accuracy: 0.6999
Epoch 2/10
574/574 [==============================] - ETA: 0s - loss: 1.2831 - accuracy: 0.5282
Epoch 00002: val_loss improved from 1.01292 to 0.90933, saving model to Tomato_model_checkpoint.h5
574/574 [==============================] - 1552s 3s/step - loss: 1.2831 - accuracy: 0.5282 - val_loss: 0.9093 - val_accuracy: 0.7276
Epoch 3/10
574/574 [==============================] - ETA: 0s - loss: 1.2399 - accuracy: 0.5484
Epoch 00003: val_loss improved from 0.90933 to 0.77624, saving model to Tomato_model_checkpoint.h5
574/574 [==============================] - 1545s 3s/step - loss: 1.2399 - accuracy: 0.5484 - val_loss: 0.7762 - val_accuracy: 0.8072
Epoch 4/10
574/574 [

In [18]:
model.save("Model_Kaggle")

INFO:tensorflow:Assets written to: Model_Kaggle\assets


NameError: name 'keras' is not defined